In [ ]:
# this cell is only for colab execution
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# this cell is only for colab execution
%cd ../content/drive/MyDrive/SymbolicMathematics-master/

/content/drive/MyDrive/SymbolicMathematics-master


In [ ]:
ls

beam_integration.ipynb  CONTRIBUTING.md  LICENSE  ode1.pth  README.md      src/
CODE_OF_CONDUCT.md      fwd_bwd.pth      main.py  ode2.pth  split_data.py


In [ ]:
import os
import numpy as np
import sympy as sp
import torch

from src.utils import AttrDict
from src.envs import build_env
from src.model import build_modules

from src.utils import to_cuda
from src.envs.sympy_utils import simplify

In [ ]:
# trained model
#!wget https://dl.fbaipublicfiles.com/SymbolicMathematics/models/ode2.pth
model_path = 'ode2.pth'
assert os.path.isfile(model_path)

In [ ]:
params = params = AttrDict({

    # environment parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'ode1',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # model parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': './ode2.pth',    

})

In [ ]:
# building the enviroment
env = build_env(params)
x = env.local_dict['x']
y = env.local_dict['y']
f = env.local_dict['f']
a0 = env.local_dict['a0']
a8 = env.local_dict['a8']

In [ ]:
modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

In [ ]:
# this is an easy ODE problem just for testing
ode = sp.Derivative(sp.Derivative(f(x)))+x

display(ode)

x + Derivative(f(x), (x, 2))

In [ ]:
ode_prefix = env.sympy_to_prefix(ode)
x1_prefix = env.clean_prefix(ode_prefix)

x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)


In [ ]:
beam_size = 10
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1, max_len=200)
    assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
print("Input function ODE:\n")
display(ode)
print("")

firstCorrect = None
for score, sent in sorted(hypotheses, key=lambda x: x[0], reverse=True):

    # parse decoded hypothesis
    ids = sent[1:].tolist()                  # decoded token IDs
    tok = [env.id2word[wid] for wid in ids]  # convert to prefix

    try:
        hyp = env.prefix_to_infix(tok)       # convert to infix
        hyp = env.infix_to_sympy(hyp)        # convert to SymPy

        # check whether we recover f if we differentiate the hypothesis
        res = "OK" if simplify(simplify(ode.subs(f(x), hyp), seconds=1), seconds=1) == 0 else "NO"
        if (firstCorrect==None and res=="OK"):
            firstCorrect = hyp

    except:
        res = "INVALID PREFIX EXPRESSION"
        hyp = tok

    # print result
    print("%.5f  %s  %s" % (score, res, hyp))
        
print("\nThe well-predicted expression with the highest score is:\n")
display(firstCorrect)

Input function ODE:



x + Derivative(f(x), (x, 2))


-0.18419  OK  a8 + a9*x - x**3/6 + x/6
-0.19393  OK  a8*x + a9 - x**3/6 + x/6
-0.20034  OK  a8 + a9*x - x**3/6
-0.20607  OK  a8*x + a9 - x**3/6
-0.20671  OK  a9 + x*(a8 - x**2)/6
-0.23822  OK  a8 - x*(a9 + x**2)/6
-0.23974  OK  a8 + a9*x - x**3/6 + 5*x/6
-0.24163  OK  a8*x + a9 - x**3/6 + 5*x/6
-0.24420  OK  a9 - x*(a8 + x**2)/6 + x
-0.24784  OK  a8 - x*(a9 + x**2)/6 + x

The well-predicted expression with the highest score is:



a8 + a9*x - x**3/6 + x/6